In [13]:
%run "../PyUtils/common.ipynb"

fileName="data/kimj/kimj.csv";
dfOriginal = LoadDataSet(fileName, columns=None, checkForDateTime=False);
dfOriginal

<IPython.core.display.Javascript object>

,cname,ampm,timeofday,dateofmeasure,pef1,pef2,pef3,pefmax,indexpef,so2,co,o3,no2,pm10,so2poi,copoi,o3poi,no2poi,pm10poi,temperaturec,rainfallmm,windspeedms,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperaturecpoi,rainfallmmpoi,windspeedmspoi,precipitationpercentpoi,vaporpressurehpapoi,dewpointtemperaturecpoi,airpressurehpapoi,sealevelpressurehpapoi,groundtemperaturecpoi,winddirection,winddirectionpoi,tmax,amax,pmax,tmin,amin,pmin,tmode,amode,pmode,tmaxlesstmin,amaxlessamin,pmaxlesspmin,tmaxpoi,amaxpoi,pmaxpoi,tminpoi,aminpoi,pminpoi,tmodepoi,amodepoi,pmodepoi,tmaxlesstminpoi,amaxlessaminpoi,pmaxlesspminpoi
0,kimj,am,5:47:00,2/5/15,121.000,98.000,112.000,121.000,R,0.014,0.800,0.032,0.022,103.000,0.010,0.800,0.023,0.032,98.000,-1.900,nan,1.800,68,3.600,-7.000,1018.000,1022.400,-1.800,-3.100,nan,2.800,67,3.300,-8.300,1011.200,1022.200,40.000,320,250,5.100,1020.900,99,-4.400,1000.000,36,0.300,1017.900,98,9.500,20.900,63,5.400,1014.000,87,-3.100,1000.000,26,-3.100,1011.000,69,8.500,14.000,61
1,kimj,am,6:09:00,2/6/15,85.000,75.000,68.000,85.000,R,0.005,0.700,0.011,0.041,62.000,0.007,0.700,0.012,0.041,58.000,-4.700,nan,0.500,93,4.000,-5.600,1021.900,1026.300,-2.800,-4.000,nan,1.200,65,3.000,-9.600,1015.100,1026.200,nan,110,290,4.700,1021.900,91,-4.400,1000.000,42,1.200,1021.500,45,9.100,21.900,49,4.700,1015.000,73,-3.800,1000.000,39,-3.400,1014.900,61,8.500,15.000,34
2,kimj,am,6:34:00,2/7/15,113.000,123.000,114.000,123.000,R,0.008,1.200,0.002,0.061,45.000,0.008,0.900,0.003,0.059,42.000,-1.900,nan,0.800,91,4.900,-3.100,1018.900,1023.300,-1.300,-1.500,nan,0.500,75,4.100,-5.300,1011.900,1022.800,nan,200,360,6.100,1023.000,94,-4.700,1000.000,44,-1.900,1021.400,92,10.800,23.000,50,6.200,1016.100,77,-4.200,1000.000,37,-4.200,1015.100,64,10.400,16.100,40
3,kimj,am,9:26:00,2/8/15,98.000,104.000,128.000,128.000,R,0.003,0.300,0.028,0.004,175.000,0.004,0.300,0.020,0.015,134.000,-9.800,nan,4.600,33,1.000,-23.000,1022.700,1027.200,-4.100,-11.600,nan,4.900,33,0.800,-24.600,1015.600,1027.000,42.000,320,290,5.100,1022.200,86,-10.400,1000.000,32,5.100,1015.600,32,15.500,22.200,54,5.200,1014.900,78,-11.900,1000.000,26,0.000,1011.800,66,17.100,14.900,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,kimj,pm,23:16:00,10/3/16,139.000,196.000,nan,196.000,Y,0.003,0.500,0.004,0.045,31.000,0.004,0.400,0.008,0.042,34.000,18.400,nan,0.700,98,20.600,18.000,1009.900,1013.900,19.500,20.200,nan,1.600,74,17.500,15.400,1003.700,1013.700,nan,290,270,27.700,1009.600,100,19.600,1000.000,54,23.400,1005.300,100,8.100,9.600,46,27.700,1003.500,93,20.000,996.600,43,20.500,996.600,90,7.700,6.900,50
999,kimj,pm,21:56:00,10/4/16,nan,nan,nan,nan,NaN,0.002,0.500,0.008,0.052,36.000,0.004,0.400,0.006,0.038,43.000,23.700,nan,4.700,59,17.300,15.200,1010.200,1014.100,20.300,22.200,nan,3.300,60,16.000,14.000,1004.000,1013.900,nan,110,70,28.200,1012.700,100,14.200,1000.000,43,27.000,1009.400,100,14.000,12.700,57,28.900,1006.700,85,15.600,1000.000,33,15.600,1004.100,39,13.300,6.700,52
1000,kimj,pm,21:45:00,10/5/16,103.000,97.000,110.000,110.000,R,0.003,0.500,0.030,0.028,47.000,0.005,0.400,0.017,0.031,55.000,19.500,nan,0.500,93,21.000,18.300,1009.900,1013.900,19.100,20.900,nan,2.300,68,16.700,14.700,1003.500,1013.400,nan,200,320,26.500,1010.300,95,17.200,1000.000,53,19.900,1004.000,67,9.300,10.300,42,26.700,1004.500,71,18.700,997.500,46,19.100,999.200,66,8.000,7.000,25
1001,kimj,pm,20:46:00,10/6/16,113.000,123.000,nan,123.000,R,0.003,0.400,0.012,0.042,38.000,0.005,0.400,0.002,0.049,38.000,16.300,nan,0.000,71,13.100,11.000,1018.200,1022.300,16.800,18.800,nan,2.000,55,11.900,9.500,1011.900,1022.000,nan,0,290,23.300,1017.900,93,13.800,1000.000,44,19.500,1017.800,84,9.500,17.900,49,23.300,1011.600,79,13.600,1000

In [158]:
df=dfOriginal.copy()
df=df.fillna(0)
dttm = df.dateofmeasure + " " + df.timeofday
df=df[ (df.pef1 > 0) & (df.pef2 > 0) & (df.pef3 >0)]

pavg = (df.pef1 + df.pef2 + df.pef3)/3
df.insert(4, 'dttm', dttm)
df.dttm = pd.to_datetime(df.dttm)
df.sort_values(by='dttm', ascending=True, inplace=True)
df.insert(4, 'pefavg', pavg)
df.insert(4, 'pefavg_pred', df.pefavg)
df.iloc[-10:, df.columns.get_loc('pefavg_pred')] = ''
df=df.drop(['timeofday', 'dateofmeasure', 'cname', 'ampm', 'pef1', 'pef2', 'pef3', 'pefmax', 'indexpef'], axis=1)
df=df.loc[df.pefavg>0]
df=df.reindex()

df.to_csv("data/kimj/combined.csv", sep=';')
df.to_csv("data/kimj/combined1.csv", sep=',')

ddf=df[df.dttm > '07-10-2016']
ddf.to_csv("data/kimj/combined-07.csv", sep=';')


In [151]:
#ts(time(df.dateofmeasure))
#plt.plot(time(df.dateofmeasure), df.pefavg)
#df[df.pefavg<60]
#df.hist("pefavg")
#df.iloc[-10:, df.columns.get_loc('pefavg_pred')] = 0
df

,pefavg_pred,pefavg,dttm,so2,co,o3,no2,pm10,so2poi,copoi,o3poi,no2poi,pm10poi,temperaturec,rainfallmm,windspeedms,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperaturecpoi,rainfallmmpoi,windspeedmspoi,precipitationpercentpoi,vaporpressurehpapoi,dewpointtemperaturecpoi,airpressurehpapoi,sealevelpressurehpapoi,groundtemperaturecpoi,winddirection,winddirectionpoi,tmax,amax,pmax,tmin,amin,pmin,tmode,amode,pmode,tmaxlesstmin,amaxlessamin,pmaxlesspmin,tmaxpoi,amaxpoi,pmaxpoi,tminpoi,aminpoi,pminpoi,tmodepoi,amodepoi,pmodepoi,tmaxlesstminpoi,amaxlessaminpoi,pmaxlesspminpoi
0,110.333,110.333,2015-02-05 05:47:00,0.014,0.800,0.032,0.022,103.000,0.010,0.800,0.023,0.032,98.000,-1.900,0.000,1.800,68,3.600,-7.000,1018.000,1022.400,-1.800,-3.100,0.000,2.800,67,3.300,-8.300,1011.200,1022.200,40.000,320,250,5.100,1020.900,99,-4.400,1000.000,36,0.300,1017.900,98,9.500,20.900,63,5.400,1014.000,87,-3.100,1000.000,26,-3.100,1011.000,69,8.500,14.000,61
1,76.000,76.000,2015-02-06 06:09:00,0.005,0.700,0.011,0.041,62.000,0.007,0.700,0.012,0.041,58.000,-4.700,0.000,0.500,93,4.000,-5.600,1021.900,1026.300,-2.800,-4.000,0.000,1.200,65,3.000,-9.600,1015.100,1026.200,0.000,110,290,4.700,1021.900,91,-4.400,1000.000,42,1.200,1021.500,45,9.100,21.900,49,4.700,1015.000,73,-3.800,1000.000,39,-3.400,1014.900,61,8.500,15.000,34
2,116.667,116.667,2015-02-07 06:34:00,0.008,1.200,0.002,0.061,45.000,0.008,0.900,0.003,0.059,42.000,-1.900,0.000,0.800,91,4.900,-3.100,1018.900,1023.300,-1.300,-1.500,0.000,0.500,75,4.100,-5.300,1011.900,1022.800,0.000,200,360,6.100,1023.000,94,-4.700,1000.000,44,-1.900,1021.400,92,10.800,23.000,50,6.200,1016.100,77,-4.200,1000.000,37,-4.200,1015.100,64,10.400,16.100,40
3,110.000,110.000,2015-02-08 09:26:00,0.003,0.300,0.028,0.004,175.000,0.004,0.300,0.020,0.015,134.000,-9.800,0.000,4.600,33,1.000,-23.000,1022.700,1027.200,-4.100,-11.600,0.000,4.900,33,0.800,-24.600,1015.600,1027.000,42.000,320,290,5.100,1022.200,86,-10.400,1000.000,32,5.100,1015.600,32,15.500,22.200,54,5.200,1014.900,78,-11.900,1000.000,26,0.000,1011.800,66,17.100,14.900,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,,119.333,2016-09-22 21:16:00,0.004,0.600,0.004,0.077,61.000,0.004,0.500,0.036,0.043,58.000,20.100,0.000,1.100,72,16.800,14.800,1013.800,1017.800,20.100,21.300,0.000,1.200,50,12.600,10.400,1007.400,1017.400,0.000,290,290,25.500,1016.800,68,17.700,1000.000,40,17.900,1016.000,60,7.800,16.800,28,27.000,1010.600,59,17.500,1000.000,31,17.700,1010.500,54,9.500,10.600,28
995,,109.667,2016-09-30 22:13:00,0.005,0.700,0.002,0.064,52.000,0.004,0.500,0.017,0.050,39.000,19.300,0.000,0.900,90,20.100,17.600,1012.200,1016.200,19.800,20.000,0.000,0.600,68,15.900,13.900,1005.900,1015.900,40.000,160,270,23.100,1013.600,88,17.800,1000.000,66,18.300,1012.300,77,5.300,13.600,22,23.400,1007.300,74,17.700,1000.000,56,23.400,1006.000,68,5.700,7.300,18
997,,126.000,2016-10-02 23:01:00,0.002,0.400,0.011,0.018,24.000,0.004,0.300,0.006,0.018,18.000,21.600,0.000,2.800,98,25.200,21.200,1004.800,1008.700,19.800,21.500,0.000,3.100,82,20.900,18.200,998.000,1007.900,1.000,160,180,20.100,1015.000,99,16.900,1000.000,83,18.600,1014.500,93,3.200,15.000,16,20.400,1008.700,93,16.700,998.300,75,18.600,1006.900,76,3.700,10.400,18
1000,,103.333,2016-10-05 21:45:00,0.003,0.500,0.030,0.028,47.000,0.005,0.400,0.017,0.031,55.000,19.500,0.000,0.500,93,21.000,18.300,1009.900,1013.900,19.100,20.900,0.000,2.300,68,16.700,14.700,1003.500,1013.400,0.000,200,320,26.500,1010.300,95,17.200,1000.000,53,19.900,1004.000,67,9.300,10.300,42,26.700,1004.500,71,18.700,997.500,46,19.100,999.200,66,8.000,7.000,25


In [157]:
ddf=df[df.dttm > '07-10-2016']
HTML(ddf.to_html())

,pefavg_pred,pefavg,dttm,so2,co,o3,no2,pm10,so2poi,copoi,o3poi,no2poi,pm10poi,temperaturec,rainfallmm,windspeedms,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,temperaturecpoi,rainfallmmpoi,windspeedmspoi,precipitationpercentpoi,vaporpressurehpapoi,dewpointtemperaturecpoi,airpressurehpapoi,sealevelpressurehpapoi,groundtemperaturecpoi,winddirection,winddirectionpoi,tmax,amax,pmax,tmin,amin,pmin,tmode,amode,pmode,tmaxlesstmin,amaxlessamin,pmaxlesspmin,tmaxpoi,amaxpoi,pmaxpoi,tminpoi,aminpoi,pminpoi,tmodepoi,amodepoi,pmodepoi,tmaxlesstminpoi,amaxlessaminpoi,pmaxlesspminpoi
926,98.000,98.000,2016-07-10 22:53:00,0.003,0.400,0.020,0.031,30.000,0.003,0.500,0.010,0.045,29.000,25.600,0.000,0.200,72,23.500,20.100,1001.200,1005.100,24.800,27.200,0.000,2.400,64,22.900,19.700,995.000,1004.600,0.000,0,270,32.800,1002.200,97,22.000,999.200,52,23.100,1001.700,90,10.800,3.000,45,32.700,995.900,83,23.200,992.900,45,29.700,995.300,70,9.500,3.000,38
927,143.000,143.000,2016-07-11 21:00:00,0.008,0.500,0.048,0.046,64.000,0.007,0.600,0.000,0.035,55.000,27.400,0.000,1.300,67,24.400,20.700,999.100,1002.900,26.600,28.600,0.000,1.500,65,25.300,21.300,992.900,1002.400,0.000,290,290,33.400,1001.600,96,23.100,998.300,46,23.300,1001.200,64,10.300,3.300,50,33.200,995.500,81,24.900,992.500,44,25.000,995.000,64,8.300,3.000,37
430,123.333,123.333,2016-07-12 05:31:00,0.005,0.400,0.013,0.028,30.000,0.005,0.400,0.000,0.020,30.000,21.900,0.100,1.200,100,26.100,21.800,999.200,1003.100,23.800,22.600,0.000,1.600,87,23.800,20.300,992.900,1002.600,1901.000,110,110,33.400,1001.600,100,21.400,998.300,46,32.800,999.900,64,12.000,3.300,54,33.200,995.500,89,22.900,992.500,44,25.000,993.200,89,10.300,3.000,45
431,119.000,119.000,2016-07-13 07:50:00,0.003,0.300,0.003,0.020,31.000,0.003,0.300,0.000,0.019,23.000,22.900,0.000,0.500,98,27.200,22.500,997.600,1001.500,23.400,23.400,0.000,2.000,84,24.100,20.500,991.600,1001.300,0.000,290,250,29.200,1000.000,100,22.300,996.700,66,28.400,997.600,69,6.900,3.300,34,30.500,993.900,86,22.300,990.700,50,23.300,991.500,54,8.200,3.200,36
929,103.333,103.333,2016-07-13 22:15:00,0.003,0.300,0.006,0.032,23.000,0.004,0.400,0.000,0.039,31.000,24.000,0.000,2.400,96,28.600,23.300,998.400,1002.200,24.100,26.000,0.000,2.200,75,25.200,21.200,991.900,1001.500,0.000,270,270,29.200,998.400,99,22.300,996.700,59,27.600,997.400,87,6.900,1.700,40,29.300,992.400,84,23.300,990.400,53,29.300,991.500,66,6.000,2.000,31
432,112.000,112.000,2016-07-14 10:54:00,0.002,0.400,0.032,0.022,36.000,0.003,0.400,0.000,0.015,26.000,28.000,0.000,1.800,67,25.200,21.200,997.200,1001.000,34.400,27.500,0.000,1.700,60,22.000,19.000,991.200,1000.800,0.000,340,320,29.200,998.400,100,23.400,996.800,59,23.700,997.500,98,5.800,1.600,41,29.300,992.000,88,24.000,990.400,53,24.600,991.300,66,5.300,1.600,35
930,83.000,83.000,2016-07-14 22:59:00,0.004,0.500,0.030,0.041,46.000,0.004,0.400,0.000,0.041,37.000,25.900,0.000,1.200,80,26.600,22.100,1000.000,1003.900,25.100,28.100,0.000,1.000,63,23.800,20.300,994.000,1003.600,0.000,180,180,30.000,999.400,100,23.400,996.800,58,23.700,997.100,98,6.600,2.600,42,31.900,993.200,88,24.000,990.500,48,30.100,991.300,58,7.900,2.700,40
433,130.000,130.000,2016-07-15 08:15:00,0.002,0.300,0.020,0.024,17.000,0.003,0.400,0.000,0.026,12.000,25.300,0.000,2.000,65,20.900,18.200,1001.600,1005.500,25.500,24.800,0.000,3.800,58,18.100,15.900,995.700,1005.400,0.000,90,50,30.000,1001.500,88,23.700,996.800,58,24.300,1001.000,67,6.300,4.700,30,31.900,995.600,76,23.800,990.800,48,24.300,991.300,58,8.100,4.800,28
932,103.000,103.000,2016-07-16 22:48:00,0.002,0.400,0.030,0.016,15.000,0.003,0.400,0.000,0.022,11.000,22.000,0.100,3.800,100,26.300,21.900,997.600,1001.500,22.200,21.600,0.000,1.400,94,24.100,20.500,991.300,1001.100,1.000,160,70,26.800,998.600,100,20.200,994.700,64,22.100,995.300,100,6.600,3.900,36,25.700,992.700,95,19.800,988.900,62,21.100,989.600,94,5.900,3.800,33
435,122.000,122.000,2016-07-17 11:03